<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_appsDev_CustomApp_opsmps3copy.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Agnostic App  📒
***Create your own Tapis App***

by Silvia Mazzoni, DesignSafe, 2025

This is a step-by-step guide to help you write your own **Tapis v3 App** — from defining the app, to registering it, and running it. 

This is a practical walkthrough for defining and deploying an **HPC** app using the Tapis v3 API.

In this notebook we will modify the opensees-mp-s3 for OpenSeesMP or OpenSeesSP, etc.

## Workflow

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 2a. Zip *tapisjob_app.sh* | It needs a zip file!                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |

In [1]:
pwd

'/home/jupyter/MyData/_ToCommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks'

In [2]:
import json

In [3]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


---
## Connect to Tapis

In [4]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-20T01:34:27+00:00
 Token expires in: 3:27:06.390424
-- LOG IN SUCCESSFUL! --


---
### Get username

In [5]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['get_tapis_username.py'])

In [6]:
username = OpsUtils.get_tapis_username(t)
print('username',username)

username silvia


In [7]:
## Define actions!

In [8]:
# Make these True once you are done validating
do_makeApp = False
do_submitJob = True

---
## Configure App

In [9]:
app_id = 'agnostic-app-test'
app_description = 'Run any Executable'

### AutoIncrement app_version

In [10]:
if do_makeApp:
    latest_app_version = OpsUtils.get_latest_app_version(t,app_id)
    print('now latest_app_version',latest_app_version)
    app_version = OpsUtils.bump_app_version(latest_app_version,'patch')
    print('now app_version',app_version)

### Where to store the files

In [11]:
if do_makeApp:
    app_system_id="designsafe.storage.default"
    app_path = f"{username}/apps/{app_id}/{app_version}"
    container_filename = f'{app_id}.zip'

### Define folder structure: app_id/version

In [12]:
if do_makeApp:
    app_folderName = f'~/MyData/myAuthoredTapisApps/{app_id}/{app_version}'; # your choice

In [13]:
if do_makeApp:
    app_folder = os.path.abspath(os.path.expanduser(app_folderName))
    os.makedirs(app_folder, exist_ok=True)
    print(f'app_folder: {app_folder}\n exists:',os.path.exists(app_folder))

---
## Create the App Files

A Tapis app needs the following **core files**:

* **Readme.MD** – App Description (OPTIONAL)<br>
This file is helpful in communicating content to the app user.  

* **app.json** – App Definition<br>
Defines the app’s metadata, inputs, parameters, and execution configuration.    

* **profile.json** – Environment Setup *(optional but common)*<br>
Loads modules or sets environment variables on the execution system. This file will load modules before your script runs. This is executed on the compute node.

* **tapisjob_app.sh** – Wrapper Script<br>
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [14]:
# let's collect the list of files we are creating as we will need this list in upload
if do_makeApp:
    appFilesList = [];

#### **Readme.MD** – App Description
This file is helpful in communicating content to the app user.  

In [15]:
if do_makeApp:
    thisFilename = 'ReadMe.MD'
    thisText = """\
    # making an application-agnostic app
    """
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **app.json** – App Definition
Defines the app’s metadata, inputs, parameters, and execution configuration.    

In [16]:
if do_makeApp:
    thisFilename = 'app.json'
    thisText = """\
    {
      "id": "__app_id__",
      "version": "__app_version__",
      "description": "__app_description__",
      "owner": "${apiUserId}",
      "enabled": true,
      "runtime": "ZIP",
      "runtimeVersion": null,
      "runtimeOptions": null,
      "containerImage": "__container_filename__",
      "jobType": "BATCH",
      "maxJobs": -1,
      "maxJobsPerUser": -1,
      "strictFileInputs": true,
      "jobAttributes": {
        "execSystemConstraints": null,
        "execSystemId": "stampede3",
        "execSystemExecDir": "${JobWorkingDir}",
        "execSystemInputDir": "${JobWorkingDir}",
        "execSystemOutputDir": "${JobWorkingDir}",
        "execSystemLogicalQueue": "skx",
        "archiveSystemId": "stampede3",
        "archiveSystemDir": "HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}",
        "archiveOnAppError": true,
        "isMpi": false,
        "mpiCmd": null,
        "parameterSet": {
          "appArgs": [
            {
              "name": "Main Program",
              "description": "Choose the binary Executable to use.",
              "arg": "OpenSeesMP",
              "inputMode": "REQUIRED",
                "notes": {
                    "enum_values": [
                        {
                            "OpenSeesMP": "OpenSeesMP"
                        },
                        {
                            "OpenSeesSP": "OpenSeesSP"
                        },
                        {
                            "python": "python"
                        }
                    ]
                }
            },
            {
              "name": "Main Script",
              "description": "The filename only of the input script to submit to executable. This file should reside in the Input Directory specified.",
              "arg": null,
              "inputMode": "REQUIRED",
              "notes": {
                "inputType": "fileInput"
              }
            },
            {
              "name": "CommandLine Arguments",
              "description": "Command-Line Arguments (Optional)",
              "arg": null,
              "inputMode": "INCLUDE_ON_DEMAND",
              "notes": {
              }
            }
            
          ],
          "containerArgs": [],
          "schedulerOptions": [
            {
              "name": "OpenSees TACC Scheduler Profile",
              "description": "Scheduler profile for the default version of OpenSees",
              "inputMode": "FIXED",
              "arg": "--tapis-profile OpenSees_default",
              "notes": {
                "isHidden": true
              }
            }
          ],
          "envVariables": [
                  { "key": "zipFileIn", "value": "", "inputMode": "INCLUDE_BY_DEFAULT" },
                  { "key": "zipFolderOut", "value": "False", "inputMode": "INCLUDE_BY_DEFAULT" },
                  { "key": "requirementFile", "value": "", "inputMode": "INCLUDE_BY_DEFAULT" }
             ],
          "archiveFilter": {
            "includes": [],
            "excludes": [],
            "includeLaunchFiles": true
          }
        },
        "fileInputs": [
          {
            "name": "Input Directory",
            "inputMode": "REQUIRED",
            "sourceUrl": null,
            "targetPath": "inputDirectory",
            "envKey": "inputDirectory",
            "description": "Input directory that includes the input script as well as any other required files. Example input is in tapis://designsafe.storage.community/app_examples/opensees/OpenSeesMP",
            "notes": {
              "selectionMode": "directory"
            }
          }
        ],
        "fileInputArrays": [],
        "nodeCount": 1,
        "coresPerNode": 48,
        "memoryMB": 192000,
        "maxMinutes": 120,
        "subscriptions": [],
        "tags": []
      },
      "tags": [
        "portalName: DesignSafe",
        "portalName: CEP"
      ],
      "notes": {
        "label": "__app_id__",
        "helpUrl": "",
        "hideNodeCountAndCoresPerNode": false,
        "isInteractive": false,
        "icon": "OpenSees",
        "category": "Simulation"
      }
    }
    """
    thisText = thisText.replace("__app_id__", app_id)
    thisText = thisText.replace("__app_version__", app_version)
    thisText = thisText.replace("__app_description__", app_description)
    thisText = thisText.replace("__container_filename__", f"tapis://{app_system_id}/{app_path}/{container_filename}")
    
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **profile.json** – Environment Setup
This file will load modules before your script runs. This is executed on the compute node.

In [17]:
if do_makeApp:
    thisFilename = 'profile.json'
    thisText = """\
    {
        "name": "__app_id__",
        "description": "Modules to load",
        "moduleLoads": [ ],
        "hiddenOptions": [
            "MEM"
        ]
    }
    """    
    # thisText = """\
    # {
    #     "name": "AgnosticApp",
    #     "description": "Modules to load",
    #     "moduleLoads": [
    #         {
    #             "modulesToLoad": [
    #                 "hdf5/1.14.4",
    #                 "opensees"
    #             ],
    #             "moduleLoadCommand": "module load"
    #         }
    #     ],
    #     "hiddenOptions": [
    #         "MEM"
    #     ]
    # }
    # """
    thisText = thisText.replace("__app_id__", app_id)
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **tapisjob_app.sh** – Wrapper Script
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [18]:
if do_makeApp:
    import textwrap, time
    from zipfile import ZipFile, ZIP_DEFLATED, ZipInfo
    
    thisFilename_sh = "tapisjob_app.sh"
    thisFilename = container_filename  # existing var


    bash_script = textwrap.dedent("""\
        #!/bin/bash
        set -euo pipefail
        set -x

        # ---- required args ----
        BINARYNAME="${1:?missing binary name}"
        INPUTSCRIPT0="${2:?missing input script}"
        shift 2

        # ---- optional env with safe defaults ----
        inputDirectory="${inputDirectory:?inputDirectory not set}"
        zipFileIn="${zipFileIn:-}"
        zipFolderOut="${zipFolderOut:-}"
        echo "zipFileIn: ${zipFileIn}"
        echo "zipFolderOut: ${zipFolderOut}"

        INPUTSCRIPT="${INPUTSCRIPT0##*/}"
        echo "INPUTSCRIPT: $INPUTSCRIPT"
        echo "ARGS: $*"

        cd -- "$inputDirectory"

        # ---- expand input ZIP if provided ----
        if [[ -n "$zipFileIn" && -f "$zipFileIn" ]]; then
          echo "Unzipping $zipFileIn ..."
          unzip -o -q "$zipFileIn"
        fi

        # ---- modules for OpenSees family ----
        case "$BINARYNAME" in
          OpenSees|OpenSeesMP|OpenSeesSP)
            module load hdf5/1.14.4
            module load opensees
            module list || true
            echo "Loaded OpenSees modules"
            ;;
        esac

        # ---- choose launcher ----
        if [[ "$BINARYNAME" == "OpenSees" ]]; then
          LAUNCH=()        # direct run for sequential
        else
          LAUNCH=(ibrun)   # MPI/SP use ibrun on TACC
        fi

        echo "Running: ${LAUNCH[*]} $BINARYNAME $INPUTSCRIPT $*"
        if ! "${LAUNCH[@]}" "$BINARYNAME" "$INPUTSCRIPT" "$@"; then
          rc=$?
          echo "Program exited with error status: $rc" >&2
          exit "$rc"
        fi
        echo "done, now zippit!"
        # ---- optional re-pack an output folder ----
        if [[ "${zipFolderOut:-}" =~ ^([Tt][Rr][Uu][Ee]|1|[Yy][Ee]?[Ss]?)$ ]]; then
          cd ..
          echo "Zipping output folder "$inputDirectory" -> ${zipFolderOut}.zip"
          zip -r -q "inputDirectory.zip" "$inputDirectory"
          rm -rf "$inputDirectory"
        fi
    """)


    zip_path = os.path.join(app_folder, thisFilename)
    print('zip_path',zip_path)

    # write with executable permission inside the ZIP
    zi = ZipInfo(thisFilename_sh)
    zi.date_time = time.localtime(time.time())[:6]
    zi.compress_type = ZIP_DEFLATED
    zi.external_attr = 0o100755 << 16   # -rwxr-xr-x on a regular file
    
    with ZipFile(zip_path, "w", ZIP_DEFLATED) as z:
        z.writestr(zi, bash_script)
        
    appFilesList.append(thisFilename)

#### File Check
Look at the files we have written and check for typos or formatting errors.

In [19]:
if do_makeApp:
    print(appFilesList)
    OpsUtils.show_text_file_in_accordion(app_folder, appFilesList)

---
## Validate App Locally

In [20]:
if do_makeApp:
    OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['validate_app_folder.py'])

In [21]:
if do_makeApp:
    validation = OpsUtils.validate_app_folder(app_folder,appFilesList)
    if not validation:
        print('Validation Failed: stopping here!!!!')
        a = 3/0

---
## Deploy the App

### Make the directory inside your 'MyData' on designsafe.storage.default.
The apps in this folder are the ones that area actually uploaded.

In [22]:
if do_makeApp:
    t.files.mkdir(systemId=app_system_id, path=app_path)
    print('app_path',app_path)

### Upload files to your deployment system (e.g., DesignSafe default storage)
Using Tapipy (Python SDK) in a Jupyter Notebook

In [23]:
if do_makeApp:
    for fname in appFilesList:
        fpath = f'{app_folder}/{fname}'
        t.upload(source_file_path=fpath,
                 system_id=app_system_id,
                 dest_file_path=f'{app_path}/{fname}')

#### Verify upload

In [24]:
if do_makeApp:
    print('app_system_id:',app_system_id)
    print('app_path:',app_path)
    appfiles = t.files.listFiles(systemId=app_system_id, path=app_path)
    for thisF in appfiles:
        print(thisF)
        print('')

---
## Register the App
This creates the actual App record that Jobs can run.

Do this Using Tapipy (Python)

In [25]:
if do_makeApp:
    # Create (or create a new version) of the app
    with open(f'{app_folder}/app.json') as f:
        app_def = json.load(f)
    t.apps.createAppVersion(**app_def)

### Check that app is up

#### List all apps

In [26]:
listType = 'ALL' # Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
select = 'id,created,description,version,owner' # Attributes to return in each result.
orderBy = 'created(asc)'
results = t.apps.getApps( orderBy=orderBy,
                         select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

--

created: 2024-12-13T20:15:33.014747Z
description: Run shell commands on remote systems
id: shell-runner-1.0.0
owner: silvia
version: 1.0.0
--

created: 2024-12-14T15:49:21.510206Z
description: Run shell commands on remote systems
id: shell-runner
owner: silvia
version: 1.0.0
--

created: 2025-08-16T16:51:35.355715Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia
owner: silvia
version: latest
--

created: 2025-08-16T18:53:58.422294Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia-new
owner: silvia
version: latest
--

created: 2025-08-17T22:34:01.193589Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-copy-silvia
owner: silvia
vers

#### List the new app

In [27]:
appMetaData = t.apps.getAppLatestVersion(appId=app_id)
print(appMetaData)


containerImage: tapis://designsafe.storage.default/silvia/apps/agnostic-app-test/0.0.18/agnostic-app-test.zip
created: 2025-08-19T21:46:58.946736Z
deleted: False
description: Run any Executable
enabled: True
id: agnostic-app-test
isPublic: False
jobAttributes: 
archiveOnAppError: True
archiveSystemDir: HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}
archiveSystemId: stampede3
cmdPrefix: None
coresPerNode: 48
description: None
dtnSystemInputDir: !tapis_not_set
dtnSystemOutputDir: !tapis_not_set
dynamicExecSystem: False
execSystemConstraints: None
execSystemExecDir: ${JobWorkingDir}
execSystemId: stampede3
execSystemInputDir: ${JobWorkingDir}
execSystemLogicalQueue: skx
execSystemOutputDir: ${JobWorkingDir}
fileInputArrays: []
fileInputs: [
autoMountLocal: True
description: Input directory that includes the input script as well as any other required files. Example input is in tapis://designsafe.storage.community/app_examples/opensees/OpenSeesMP
envKey: inputDi

---
## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

---
### Initialize

In [28]:
tapisInputAll = {}

---
### SLURM-Specific Input

In [29]:
tapisInputAll["maxMinutes"] = 6

tapisInputAll["execSystemId"] = "stampede3"
tapisInputAll["execSystemLogicalQueue"] = "skx-dev"
tapisInputAll["nodeCount"] = 1
tapisInputAll["coresPerNode"] = 48
tapisInputAll["allocation"] = "DS-HPC1"

tapisInputAll['archive_system']='Work' # Options: MyData or Work

---
### App-Specific Input

In [30]:
print('app_id:',app_id)

app_id: agnostic-app-test


### OpenSees

In [31]:
def get_tapis_job_description(t, tapisInput):
    """
    Build a complete Tapis v3 job description dict from user-friendly inputs.

    This function:
      1) Resolves the Files base URL for inputs from `tapisInput["storage_system"]`
         (supports MyData/community/published; can be overridden via
         `tapisInput["storage_system_baseURL"]`).
      2) Ensures a concrete application version:
         - If `appVersion` is missing or equals "latest", it resolves a pinned
           version via `OpsUtils.get_latest_app_version(t, <appId>)`.
      3) Validates required fields (set differs for OpenSees-Express vs. HPC apps).
      4) Constructs job attributes, fileInputs, parameterSet, and archive settings.

    Auto baseURL selection
    ----------------------
    If `storage_system_baseURL` is not provided:
      - If `storage_system` contains "mydata": uses
        `tapis://designsafe.storage.default/<username>` where `<username>` is taken
        from `t.authenticator.get_userinfo().username`.
      - If `storage_system` contains "community": uses
        `tapis://designsafe.storage.community`
      - If `storage_system` contains "published": uses
        `tapis://designsafe.storage.published`
      - Else: prints a message and returns -1.

    App families and required keys
    ------------------------------
    *OpenSees-Express* (`appId == "opensees-express"`)
      Required: ['name','appId','appVersion','maxMinutes','archive_system',
                 'storage_system','input_folder','input_filename']
      - Sets `parameterSet.envVariables` with:
          mainProgram = OpenSees
          tclScript   = <input_filename>
      - `fileInputs = [{"name": "Input Directory", "sourceUrl": "<base>/<input_folder>"}]`
      - Archive options:
          archive_system == 'MyData' -> designsafe.storage.default under
              ${EffectiveUserId}/tapis-jobs-archive/${JobCreateDate}/${JobUUID}
          archive_system == 'Temp'   -> cloud.data:/tmp/${JobOwner}/tapis-jobs-archive/...

    *HPC OpenSees apps* (e.g., opensees-mp-s3, opensees-sp-*, etc.)
      Required: ['name','appId','appVersion','execSystemId','execSystemLogicalQueue',
                 'nodeCount','coresPerNode','maxMinutes','allocation','archive_system',
                 'storage_system','input_folder','input_filename']
      - Sets base job attributes (system, queue, resources).
      - `parameterSet.appArgs = [{"name": "Main Script", "arg": <input_filename>}]`
      - `parameterSet.schedulerOptions = [{"name": "TACC Allocation",
                                           "arg": f"-A {allocation}"}]`
      - `fileInputs` as above.
      - Archive options:
          archive_system == 'MyData' -> designsafe.storage.default under
              ${EffectiveUserId}/tapis-jobs-archive/${JobCreateDate}/${JobUUID}
          archive_system == 'Work'   -> exec system $WORK/tapis-jobs-archive/...

    Returns
    -------
    dict
        A fully formed Tapis job description ready for submission.
        Returns -1 if validation fails or baseURL/appVersion cannot be determined.

    Side effects
    ------------
    - Prints a list of missing required keys (if any).
    - Prints a short confirmation ("All Input is Complete") on success.

    Author
    ------
    Silvia Mazzoni, DesignSafe (silviamazzoni@yahoo.com)

    Date
    ----
    2025-08-16

    Version
    -------
    1.2
    """
    # Silvia Mazzoni, 2025
    from OpsUtils import OpsUtils  # for get_latest_app_version

    def checkRequirements(tapisInputIN, RequiredInputList):
        nmiss = 0
        for thisReq in RequiredInputList:
            if thisReq not in tapisInputIN:
                nmiss += 1
                print(f"YOU need to define the following input: {thisReq}")
        return nmiss

    # --- Resolve storage baseURL if needed ---
    if "storage_system_baseURL" not in tapisInput:
        storage_system_lower = tapisInput.get("storage_system", "").lower()
        if "mydata" in storage_system_lower:
            # Get user information for MyData path
            user_info = t.authenticator.get_userinfo()
            username = user_info.username
            tapisInput["storage_system_baseURL"] = f"tapis://designsafe.storage.default/{username}"
        elif "community" in storage_system_lower:
            tapisInput["storage_system_baseURL"] = "tapis://designsafe.storage.community"
        elif "published" in storage_system_lower:
            tapisInput["storage_system_baseURL"] = "tapis://designsafe.storage.published"
        else:
            print("Please specify tapisInput['storage_system_baseURL']!")
            return -1

    sourceUrl = f"{tapisInput['storage_system_baseURL']}/{tapisInput['input_folder']}"

    # --- Ensure a concrete appVersion (resolve 'latest' or missing) ---
    appId = tapisInput["appId"]
    print('appId',appId)
    if ("appVersion" not in tapisInput) or (str(tapisInput["appVersion"]).lower() == "latest"):
        print('iff')
        resolved = OpsUtils.get_latest_app_version(t, appId)
        print('resolved',resolved)
        if not resolved or resolved in ("none", ""):
            print(f"Unable to resolve latest version for appId='{appId}'. Please specify appVersion.")
            return -1
        tapisInput["appVersion"] = resolved

    # --- Defaults & branching ---
    # If Express, default the exec system to the Express VM unless provided
    if appId == "opensees-express" and "execSystemId" not in tapisInput:
        tapisInput["execSystemId"] = "wma-exec-01"
    if "execSystemId" not in tapisInput:
        tapisInput["execSystemId"] = "stampede3"

    job_description = {}
    nmiss = 999

    # Express (runs on wma-exec-01)
    if appId == "opensees-express":
        print('hello')
        RequiredInputList = [
            "name", "appId", "appVersion", "maxMinutes", "archive_system",
            "storage_system", "input_folder", "input_filename"
        ]
        nmiss = checkRequirements(tapisInput, RequiredInputList)
        if nmiss == 0:
            parameterSet = {}
            job_description["name"] = tapisInput["name"]
            job_description["maxMinutes"] = tapisInput["maxMinutes"]
            job_description["appId"] = tapisInput["appId"]
            job_description["appVersion"] = tapisInput["appVersion"]
            fileInputs = [{"name": "Input Directory", "sourceUrl": sourceUrl}]
            if not 'main_program' in tapisInput.keys():
                tapisInput["main_program"] = 'OpenSees'
            parameterSet["envVariables"] = [
                {"key": "mainProgram", "value": tapisInput["main_program"]},
                {"key": "tclScript", "value": tapisInput["input_filename"]},
            ]
            job_description["fileInputs"] = fileInputs
            job_description["parameterSet"] = parameterSet

            # Archive location
            if "archive_system" in tapisInput:
                if tapisInput["archive_system"] == "MyData":
                    job_description["archiveSystemId"] = "designsafe.storage.default"
                    job_description["archiveSystemDir"] = "${EffectiveUserId}/tapis-jobs-archive/${JobCreateDate}/${JobUUID}"
                elif tapisInput["archive_system"] == "Temp":
                    job_description["archiveSystemId"] = "cloud.data"
                    job_description["archiveSystemDir"] = "/tmp/${JobOwner}/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}"

    else:
        print('else')
        # HPC (e.g., OpenSeesMP/SP on Stampede3)
        RequiredInputList = [
            "name","appId","appVersion","execSystemId","execSystemLogicalQueue",
            "nodeCount","coresPerNode","maxMinutes","allocation","archive_system",
            "storage_system","input_folder","input_filename"
        ]
        nmiss = checkRequirements(tapisInput, RequiredInputList)
        if nmiss == 0:
            parameterSet = {}
            job_description["name"] = tapisInput["name"]
            job_description["execSystemId"] = tapisInput["execSystemId"]
            job_description["execSystemLogicalQueue"] = tapisInput["execSystemLogicalQueue"]
            job_description["maxMinutes"] = tapisInput["maxMinutes"]
            job_description["nodeCount"] = tapisInput["nodeCount"]
            job_description["coresPerNode"] = tapisInput["coresPerNode"]
            job_description["appId"] = tapisInput["appId"]
            job_description["appVersion"] = tapisInput["appVersion"]
            fileInputs = [{"name": "Input Directory", "sourceUrl": sourceUrl}]
            if not 'main_program' in tapisInput.keys():
                tapisInput["main_program"] = 'OpenSeesMP'
            if not 'command_line_args' in tapisInput:
                tapisInput["command_line_args"] = ''
            parameterSet["appArgs"] = [{"name": "Main Program", "arg": tapisInput["main_program"]},
                                       {"name": "Main Script", "arg": tapisInput["input_filename"]},
                                       {"name": "CommandLine Arguments", "arg": tapisInput["command_line_args"]}
                                      ]
            parameterSet["envVariables"] = []
            parameterSet["schedulerOptions"] = [{"name": "TACC Allocation", "arg": f"-A {tapisInput['allocation']}"}]
            job_description["fileInputs"] = fileInputs
            job_description["parameterSet"] = parameterSet

            # Archive location
            if "archive_system" in tapisInput:
                if tapisInput["archive_system"] == "MyData":
                    job_description["archiveSystemId"] = "designsafe.storage.default"
                    job_description["archiveSystemDir"] = "${EffectiveUserId}/tapis-jobs-archive/${JobCreateDate}/${JobUUID}"
                elif tapisInput["archive_system"] == "Work":
                    job_description["archiveSystemId"] = tapisInput["execSystemId"]
                    job_description["archiveSystemDir"] = "HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}"
                    
    if nmiss == 0:
        for envVar in ['zipFileIn','zipFolderOut']:
            if envVar in tapisInput:
                job_description["parameterSet"]['envVariables'][envVar].append(tapisInput[envVar])
                

    # --- Finalize ---
    if nmiss > 0:
        print("Please resubmit with all required input")
        return -1
    else:
        print("All Input is Complete")
        return job_description


In [32]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['main_program'] = 'OpenSees'
tapisInput['input_filename'] = 'Ex1a.Canti2D.Push.tcl'

tapisInput['command_line_args'] = '[33,22]'

# -----------------------------------------------------
if do_submitJob:
    for hereApp in [app_id,'openSees-express']:
        print(f\n -- {hereApp} --\n')
        here_tapisInput = tapisInput.copy()
        here_tapisInput["appId"] = hereApp 
        here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"]
        print('here_tapisInput',here_tapisInput)
        print(get_tapis_job_description(t, here_tapisInput))
        # jobReturns = OpsUtils.run_tapis_job(t,tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

/n -- agnostic-app-test --

here_tapisInput {'maxMinutes': 6, 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'nodeCount': 1, 'coresPerNode': 48, 'allocation': 'DS-HPC1', 'archive_system': 'Work', 'storage_system': 'CommunityData', 'input_folder': 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples', 'main_program': 'OpenSees', 'input_filename': 'Ex1a.Canti2D.Push.tcl', 'command_line_args': '[33,22]', 'appId': 'agnostic-app-test', 'name': 'OpenSees_agnostic-app-test'}
appId agnostic-app-test
iff
get_latest_app_version
latest 
containerImage: tapis://designsafe.storage.default/silvia/apps/agnostic-app-test/0.0.18/agnostic-app-test.zip
created: 2025-08-19T21:46:58.946736Z
deleted: False
description: Run any Executable
enabled: True
id: agnostic-app-test
isPublic: False
jobAttributes: 
archiveOnAppError: True
archiveSystemDir: HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}
archiveSystemId: stampede3
cmdPr

In [33]:
a = 3/0

ZeroDivisionError: division by zero

### OpenSeesMP

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['main_program'] = 'OpenSeesMP'
tapisInput['input_filename'] = 'Ex1a_many.Canti2D.Push.mp.tcl'

tapisInput['command_line_args'] = '[33,22]'

# -----------------------------------------------------
if do_submitJob:
    for hereApp in [app_id,'opensees-mp-s3']:
        print(hereApp)
        tapisInput["appId"] = hereApp 
        tapisInput["name"] = tapisInput['main_program'] + '_' + tapisInput["appId"]
        jobReturns = OpsUtils.run_tapis_job(t,tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

### OpenSeesSP

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['main_program'] = 'OpenSeesSP'
tapisInput['input_filename'] = 'simpleSP.tcl'

tapisInput['command_line_args'] = '[33,22]'

# -----------------------------------------------------
if do_submitJob:
    for hereApp in [app_id,'opensees-sp-s3']:
        print(hereApp)
        tapisInput["appId"] = hereApp 
        tapisInput["name"] = tapisInput['main_program'] + '_' + tapisInput["appId"]
        jobReturns = OpsUtils.run_tapis_job(t,tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

### OpenSeesPy

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['main_program'] = 'python'
tapisInput['input_filename'] = 'Ex1a.Canti2D.Push.mpi.py'

tapisInput['command_line_args'] = '[33,22]'

# -----------------------------------------------------
if do_submitJob:
    for hereApp in [app_id]:
        print(hereApp)
        tapisInput["appId"] = hereApp 
        tapisInput["name"] = tapisInput['main_program'] + '_' + tapisInput["appId"]
        jobReturns = OpsUtils.run_tapis_job(t,tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

### OpenSeesPy with zipFolderOut

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['zipFolderOut'] = 'True'

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['main_program'] = 'python'
tapisInput['input_filename'] = 'Ex1a.Canti2D.Push.mpi.py'

tapisInput['command_line_args'] = '[33,22]'

# -----------------------------------------------------
if do_submitJob:
    for hereApp in [app_id]:
        print(hereApp)
        tapisInput["appId"] = hereApp 
        tapisInput["name"] = tapisInput['main_program'] + '_' + tapisInput["appId"]
        jobReturns = OpsUtils.run_tapis_job(t,tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------